# SQL in Python

In this example, we'll connect to an SQLite3 database—SQLite is a light-weight DMBS. This exact code may not work with other DBMSs (e.g., MySQL, SQL Server, Oracle, etc.), but something similar should work. Different databases support different flavors of SQL (column types are a big difference), but again, the example below should work with other DBMSs with slight modifications.

This demonstration uses a benchmark/testing database called Chinook that consists of data pulled from iTunes. [Check out the GitHub page if you want to learn more](https://github.com/lerocha/chinook-database).

Here is the entity-relation diagram (ERD):

![Picture title](chinook-erd.png)

In [2]:
import sqlite3
# If you don't have this, install with pip: python3 -m pip install sqlite3

# Step 1, make a connection; for sqlite3, we need the name of the database file.
connection = sqlite3.connect("../data/Chinook_Sqlite.sqlite")

# Step 2, get a cursor.
cursor = connection.cursor()

# Step 3, execute SQL statement(s).
cursor.execute("select * from Album limit 10")
# -- for select statements, do something with the results using
#   * cursor.fetchone() -- get the first result
#   * cursor.fetchmany(n) -- get a list of the first n results
#   * cursor.fetchall() -- get a list of all the results
cursor.fetchall()


# Step 4, commit -- ONLY IF MODIFYING THE DATABASE; skip this step for SELECTs.
# connection.commit()

# Step 5, close the connection when you're done.
# connection.close()



[(1, 'For Those About To Rock We Salute You', 1),
 (2, 'Balls to the Wall', 2),
 (3, 'Restless and Wild', 2),
 (4, 'Let There Be Rock', 1),
 (5, 'Big Ones', 3),
 (6, 'Jagged Little Pill', 4),
 (7, 'Facelift', 5),
 (8, 'Warner 25 Anos', 6),
 (9, 'Plays Metallica By Four Cellos', 7),
 (10, 'Audioslave', 8)]

In [3]:
# Get and print the first 10 rows of the Album table.
cursor.execute("select * from Album")
cursor.fetchmany(10)

[(1, 'For Those About To Rock We Salute You', 1),
 (2, 'Balls to the Wall', 2),
 (3, 'Restless and Wild', 2),
 (4, 'Let There Be Rock', 1),
 (5, 'Big Ones', 3),
 (6, 'Jagged Little Pill', 4),
 (7, 'Facelift', 5),
 (8, 'Warner 25 Anos', 6),
 (9, 'Plays Metallica By Four Cellos', 7),
 (10, 'Audioslave', 8)]

In [4]:
# Get just the album titles.
cursor.execute("select Title from Album")
cursor.fetchmany(10)

[('For Those About To Rock We Salute You',),
 ('Balls to the Wall',),
 ('Restless and Wild',),
 ('Let There Be Rock',),
 ('Big Ones',),
 ('Jagged Little Pill',),
 ('Facelift',),
 ('Warner 25 Anos',),
 ('Plays Metallica By Four Cellos',),
 ('Audioslave',)]

In [21]:
# (Question 1)
# Order album titles alphabetically.


[('...And Justice For All',),
 ('20th Century Masters - The Millennium Collection: The Best of Scorpions',),
 ('A Copland Celebration, Vol. I',),
 ('A Matter of Life and Death',),
 ('A Real Dead One',),
 ('A Real Live One',),
 ('A Soprano Inspired',),
 ('A TempestadeTempestade Ou O Livro Dos Dias',),
 ('A-Sides',),
 ('Ace Of Spades',)]

In [22]:
# Order album titles reverse alphabetically.


[('[1997] Black Light Syndrome',),
 ('Zooropa',),
 ('Worlds',),
 ('Weill: The Seven Deadly Sins',),
 ('Warner 25 Anos',),
 ('War',),
 ('Walking Into Clarksdale',),
 ('Wagner: Favourite Overtures',),
 ('Vs.',),
 ('Vozes do MPB',)]

In [23]:
# How many albums are in the database?


(347,)

## Joins

In [24]:
# cursor.execute("select InvoiceLine.*, Track.Name from InvoiceLine join Track")
# result = cursor.fetchall()
# print(len(result))
# result[:3]

In [25]:
cursor.execute("select InvoiceLine.*, Track.Name from InvoiceLine join Track on InvoiceLine.TrackId = Track.TrackId")
result = cursor.fetchall()
print(len(result))
result[:3]

2240


[(1, 1, 2, 0.99, 1, 'Balls to the Wall'),
 (2, 1, 4, 0.99, 1, 'Restless and Wild'),
 (3, 2, 6, 0.99, 1, 'Put The Finger On You')]

In [26]:
# How many tracks are there per album?
cursor.execute("select Album.AlbumId, count(*) \
    from Track join Album \
        on Track.AlbumId = Album.AlbumId \
    group by Album.AlbumId")
cursor.fetchmany(10)

[(1, 10),
 (2, 1),
 (3, 3),
 (4, 8),
 (5, 15),
 (6, 13),
 (7, 12),
 (8, 14),
 (9, 8),
 (10, 14)]

In [28]:
# Order these by track count, highest first
cursor.execute("select Album.AlbumId, count(*) as TrackCount  \
    from Track join Album \
        on Track.AlbumId = Album.AlbumId \
    group by Album.AlbumId \
    order by TrackCount desc")
cursor.fetchmany(10)

[(141, 57),
 (23, 34),
 (73, 30),
 (229, 26),
 (230, 25),
 (251, 25),
 (83, 24),
 (231, 24),
 (253, 24),
 (24, 23)]

In [29]:
# Find the album titles that go with the album ids.
cursor.execute("select Album.Title, Album.AlbumId, count(*) as TrackCount  \
    from Track join Album \
        on Track.AlbumId = Album.AlbumId \
    group by Album.AlbumId \
    order by TrackCount desc")
cursor.fetchmany(10)

[('Greatest Hits', 141, 57),
 ('Minha Historia', 23, 34),
 ('Unplugged', 73, 30),
 ('Lost, Season 3', 229, 26),
 ('Lost, Season 1', 230, 25),
 ('The Office, Season 3', 251, 25),
 ('My Way: The Best Of Frank Sinatra [Disc 1]', 83, 24),
 ('Lost, Season 2', 231, 24),
 ('Battlestar Galactica (Classic), Season 1', 253, 24),
 ('Afrociberdelia', 24, 23)]

In [31]:
# Find the *artist and album titles* that go with the album ids.
cursor.execute("select Artist.Name, Album.Title, Album.AlbumId, count(*) as TrackCount  \
    from Track join Album  \
        on Track.AlbumId = Album.AlbumId \
    join Artist \
        on Album.ArtistId = Artist.ArtistId \
    group by Album.AlbumId \
    order by TrackCount desc")
cursor.fetchmany(10)

[('Lenny Kravitz', 'Greatest Hits', 141, 57),
 ('Chico Buarque', 'Minha Historia', 23, 34),
 ('Eric Clapton', 'Unplugged', 73, 30),
 ('Lost', 'Lost, Season 3', 229, 26),
 ('Lost', 'Lost, Season 1', 230, 25),
 ('The Office', 'The Office, Season 3', 251, 25),
 ('Frank Sinatra', 'My Way: The Best Of Frank Sinatra [Disc 1]', 83, 24),
 ('Lost', 'Lost, Season 2', 231, 24),
 ('Battlestar Galactica (Classic)',
  'Battlestar Galactica (Classic), Season 1',
  253,
  24),
 ('Chico Science & Nação Zumbi', 'Afrociberdelia', 24, 23)]

In [35]:
aTuple = ('a', 'b', 'c')
aListVersionOfATuple = list(aTuple)
# aTuple[0] = 'z'
aListVersionOfATuple[0] = 'z'
aListVersionOfATuple

['z', 'b', 'c']

## Getting column names from an SQLite database

In [36]:
connection.row_factory = sqlite3.Row
cursor = connection.cursor()

In [38]:
# Run one of our earlier queries...

cursor.execute("select Artist.Name, Album.Title, Album.AlbumId, count(*) as TrackCount  \
    from Track join Album  \
        on Track.AlbumId = Album.AlbumId \
    join Artist \
        on Album.ArtistId = Artist.ArtistId \
    group by Album.AlbumId \
    order by TrackCount desc")
results = cursor.fetchmany(10)

In [43]:
results[0]['Name']
results[0]['Title']

'Greatest Hits'

In [44]:
from collections import namedtuple

def namedtuple_factory(cursor, row):
    fields = [column[0] for column in cursor.description]
    cls = namedtuple("Row", fields)
    return cls._make(row)

connection.row_factory = namedtuple_factory
cursor = connection.cursor()

## Run the same query again

In [45]:
cursor.execute("select Artist.Name, Album.Title, Album.AlbumId, count(*) as TrackCount  \
    from Track join Album  \
        on Track.AlbumId = Album.AlbumId \
    join Artist \
        on Album.ArtistId = Artist.ArtistId \
    group by Album.AlbumId \
    order by TrackCount desc")
results = cursor.fetchmany(10)

In [46]:
results

[Row(Name='Lenny Kravitz', Title='Greatest Hits', AlbumId=141, TrackCount=57),
 Row(Name='Chico Buarque', Title='Minha Historia', AlbumId=23, TrackCount=34),
 Row(Name='Eric Clapton', Title='Unplugged', AlbumId=73, TrackCount=30),
 Row(Name='Lost', Title='Lost, Season 3', AlbumId=229, TrackCount=26),
 Row(Name='Lost', Title='Lost, Season 1', AlbumId=230, TrackCount=25),
 Row(Name='The Office', Title='The Office, Season 3', AlbumId=251, TrackCount=25),
 Row(Name='Frank Sinatra', Title='My Way: The Best Of Frank Sinatra [Disc 1]', AlbumId=83, TrackCount=24),
 Row(Name='Lost', Title='Lost, Season 2', AlbumId=231, TrackCount=24),
 Row(Name='Battlestar Galactica (Classic)', Title='Battlestar Galactica (Classic), Season 1', AlbumId=253, TrackCount=24),
 Row(Name='Chico Science & Nação Zumbi', Title='Afrociberdelia', AlbumId=24, TrackCount=23)]

In [48]:
results[0].Name

'Lenny Kravitz'